<a href="https://colab.research.google.com/github/Kavit212/WAE-Ensemble/blob/main/WAE_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-estimator==2.2.*
!pip install tensorflow==2.2.0
!pip install keras==2.3.1
!pip install git+https://github.com/qubvel/segmentation_models


In [ ]:
!python --version

In [ ]:
# Connect to your own data source

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content


In [ ]:
!mkdir 128_patches

In [ ]:
#change file path according to your data source

!tar -xvf "/content/gdrive/MyDrive/Testing data/J558/J558L_train_images.tar" -C "/content/128_patches"
!tar -xvf "/content/gdrive/MyDrive/Testing data/J558/J558_train_masks.tar" -C "/content/128_patches"


!tar -xvf "/content/gdrive/MyDrive/Testing data/J558/J558L_test_images.tar" -C "/content/128_patches"
!tar -xvf "/content/gdrive/MyDrive/Testing data/J558/J558_test_masks.tar" -C "/content/128_patches"






In [ ]:
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir

In [ ]:
### Image Augmentation

import numpy as np
from matplotlib import pyplot as plt
from skimage.transform import AffineTransform, warp
from skimage import io, img_as_ubyte
import random
import os
from scipy.ndimage import rotate

import albumentations as A
images_to_generate=500

In [ ]:
!rm -rf aug_img
!rm -rf aug_mask

In [ ]:
!mkdir aug_img
!mkdir aug_mask

In [ ]:
# change to your own preferred data path

images_path="/content/128_patches/J558L_train_images" #path to original images
masks_path = "/content/128_patches/J558_train_masks"
img_augmented_path="/content/aug_img" # path to store aumented images
msk_augmented_path="/content/aug_mask" # path to store aumented images
images=[] # to store paths of images from folder
masks=[]

In [ ]:
for im in os.listdir(images_path):  # read image name from folder and append its path into "images" array     
    images.append(os.path.join(images_path,im))

for msk in os.listdir(masks_path):  # read image name from folder and append its path into "images" array     
    masks.append(os.path.join(masks_path,msk))


aug = A.Compose([
    A.VerticalFlip(p=1),              
    A.RandomRotate90(p=1),
    A.HorizontalFlip(p=1),
    A.Transpose(p=1),
    #A.ElasticTransform(p=0.5, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
    A.GridDistortion(p=1)
    ]
)


In [ ]:
random.seed(42)

i=1   # variable to iterate till images_to_generate


while i<=images_to_generate: 
    number = random.randint(0, len(images)-1)  #PIck a number to select an image & mask
    image = images[number]
    mask = masks[number]
    #print(image, mask)
    #image=random.choice(images) #Randomly select an image name
    original_image = io.imread(image)
    original_mask = io.imread(mask)
    
    augmented = aug(image=original_image, mask=original_mask)
    transformed_image = augmented['image']
    transformed_mask = augmented['mask']

        
    new_image_path= "%s/augmented_image_%s.png" %(img_augmented_path, i)
    new_mask_path = "%s/augmented_image_%s.png" %(msk_augmented_path, i)
    io.imsave(new_image_path, transformed_image)
    io.imsave(new_mask_path, transformed_mask)
    i =i+1
    

In [ ]:
import tensorflow as tf
from tensorflow import keras

%env SM_FRAMEWORK=tf.keras

import segmentation_models as sm

import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
#import keras 





#from tf.keras.utils import normalize 


#from tf.keras.metrics import MeanIoU

In [ ]:
#Resizing images, if needed
SIZE_X = 512
SIZE_Y = 512
#Number of classes for segmentation
n_classes = 3

In [ ]:
#Capture training image info as a list
train_images = []

for directory_path in glob.glob("/content/aug_img"):
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path, 1)       
        img = cv2.resize(img, (SIZE_Y, SIZE_X))
        train_images.append(img)
       
#Convert list to array for machine learning processing        
train_images = np.array(train_images)

#Capture mask/label info as a list
train_masks = [] 
for directory_path in glob.glob("/content/aug_mask"):
    for mask_path in glob.glob(os.path.join(directory_path, "*.png")):
        mask = cv2.imread(mask_path, 0)       
        mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST)  #Otherwise ground truth changes due to interpolation
        train_masks.append(mask)
        
#Convert list to array for machine learning processing          
train_masks = np.array(train_masks)


In [ ]:
print(train_images.shape)
print(train_masks.shape)

In [ ]:
#Capture test image info as a list
test_images = []

for directory_path in glob.glob("/content/128_patches/J558L_test_images"):
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path, 1)       
        #img = cv2.resize(img, (SIZE_Y, SIZE_X))
        test_images.append(img)
       
#Convert list to array for machine learning processing        
test_images = np.array(test_images)

#Capture mask/label info as a list
test_masks = [] 
for directory_path in glob.glob("/content/128_patches/J558_test_masks"):
    for mask_path in glob.glob(os.path.join(directory_path, "*.png")):
        mask = cv2.imread(mask_path, 0)       
        #mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST)  #Otherwise ground truth changes due to interpolation
        test_masks.append(mask)
        
#Convert list to array for machine learning processing          
test_masks = np.array(test_masks)


In [ ]:
print(test_images.shape)
print(test_masks.shape)

In [ ]:
###############################################
#Encode labels... but multi dim array so need to flatten, encode and reshape
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = train_masks.shape
train_masks_reshaped = train_masks.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

np.unique(train_masks_encoded_original_shape)


In [ ]:
###############################################
#Encode labels... but multi dim array so need to flatten, encode and reshape
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = test_masks.shape
test_masks_reshaped = test_masks.reshape(-1,1)
test_masks_reshaped_encoded = labelencoder.fit_transform(test_masks_reshaped)
test_masks_encoded_original_shape = test_masks_reshaped_encoded.reshape(n, h, w)

np.unique(test_masks_encoded_original_shape)


In [ ]:
#train_images = np.expand_dims(train_images, axis=3)
#train_images = normalize(train_images, axis=1)

#test_images = np.expand_dims(test_images, axis=3)
#test_images = normalize(test_images, axis=1)

train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)
test_masks_input = np.expand_dims(test_masks_encoded_original_shape, axis=3)

In [ ]:
#################################################
#train_images = np.expand_dims(train_images, axis=3)
#train_images = normalize(train_images, axis=1)

#train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)



X_train = train_images
y_train = train_masks_input
X_test = test_images
y_test = test_masks_input


print("Class values in the dataset are ... ", np.unique(y_train))  # 0 is the background/few unlabeled 



In [ ]:
#Sanity check, view few mages
import random
import numpy as np
image_number = random.randint(0, len(X_train))
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(X_train[image_number, :,:, 0], cmap='gray')
plt.subplot(122)
plt.imshow(np.reshape(y_train[image_number], (512,512)), cmap='gray')
plt.show()


In [ ]:

#from keras.utils import to_categorical
train_masks_cat = tf.keras.utils.to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))


test_masks_cat = tf.keras.utils.to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))

In [ ]:
#Reused parameters in all models

n_classes=3
activation='softmax'

LR = 0.0001
optim = keras.optimizers.Adam(LR)


# set class weights for dice_loss according to number of classes 
#dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.25, 0.25, 0.25, 0.25]))
dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.33, 0.33, 0.33]))  
#dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.2, 0.2, 0.2, 0.2, 0.2]))
#dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)



metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

In [ ]:
################
# Calbacks



csv_logger1 = tf.keras.callbacks.CSVLogger('/content/gdrive/MyDrive/Test_Output/Ensemble_Wanner/log_Ensemble_Resnet34-N.csv', append=False, separator=',')
csv_logger2 = tf.keras.callbacks.CSVLogger('/content/gdrive/MyDrive/Test_Output/Ensemble_Wanner/log_Ensemble_Inception-N.csv', append=False, separator=',')
csv_logger3 = tf.keras.callbacks.CSVLogger('/content/gdrive/MyDrive/Test_Output/Ensemble_Wanner/log_Ensemble_Vgg19-N.csv', append=False, separator=',')
csv_logger4 = tf.keras.callbacks.CSVLogger('/content/gdrive/MyDrive/Test_Output/Ensemble_Wanner/log_Ensemble_Seresnet-N.csv', append=False, separator=',')
csv_logger5 = tf.keras.callbacks.CSVLogger('/content/gdrive/MyDrive/Test_Output/Ensemble_Wanner/log_Ensemble_EfficientnetB4-N.csv', append=False, separator=',')
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.4, patience=10, mode='min', verbose=0)
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='min')




In [ ]:
########################################################################
###Model 1
BACKBONE1 = 'resnet34'

preprocess_input1 = sm.get_preprocessing(BACKBONE1)

# preprocess input
X_train1 = preprocess_input1(X_train)
X_test1 = preprocess_input1(X_test)

# define model
model1 = sm.Unet(BACKBONE1, encoder_weights='imagenet', classes=n_classes, activation=activation)

# compile keras model with defined optimozer, loss and metrics
model1.compile(optim, total_loss, metrics=metrics)


#print(model1.summary())


history1=model1.fit(X_train1, 
          y_train_cat,
          batch_size=8, 
          epochs=500,
          verbose=1,
          callbacks=[earlyStopping, reduce_lr, csv_logger1],
          validation_data=(X_test1, y_test_cat))


model1.save('/content/gdrive/MyDrive/Test_Output/Ensemble_Wanner/resnet34-N.hdf5')
#model1.save('platelet_resnet.hdf5')

In [ ]:
############################################################
###Model 2

BACKBONE2 = 'inceptionv3'
preprocess_input2 = sm.get_preprocessing(BACKBONE2)

# preprocess input
X_train2 = preprocess_input2(X_train)
X_test2 = preprocess_input2(X_test)

# define model
model2 = sm.Unet(BACKBONE2, encoder_weights='imagenet', classes=n_classes, activation=activation)


# compile keras model with defined optimozer, loss and metrics
model2.compile(optim, total_loss, metrics)



#print(model2.summary())


history2=model2.fit(X_train2, 
          y_train_cat,
          batch_size=8, 
          epochs=500,
          verbose=1,
          callbacks=[reduce_lr, earlyStopping, csv_logger2],
          validation_data=(X_test2, y_test_cat))


model2.save('/content/gdrive/MyDrive/Test_Output/Ensemble_Wanner/INception-N.hdf5')

In [ ]:
#####################################################
###Model 3

BACKBONE3 = 'vgg19'
#BACKBONE3 = 'seresnet34'
preprocess_input3 = sm.get_preprocessing(BACKBONE3)

# preprocess input
X_train3 = preprocess_input3(X_train)
X_test3 = preprocess_input3(X_test)


# define model
model3 = sm.Unet(BACKBONE3, encoder_weights='imagenet', classes=n_classes, activation=activation)

# compile keras model with defined optimozer, loss and metrics
model3.compile(optim, total_loss, metrics)



#print(model3.summary())

history3=model3.fit(X_train3, 
          y_train_cat,
          batch_size=8, 
          epochs=500,
          verbose=1,
          callbacks=[csv_logger3, reduce_lr, earlyStopping],
          validation_data=(X_test3, y_test_cat))


model3.save('/content/gdrive/MyDrive/Test_Output/Ensemble_Wanner/VGG19-N.hdf5')

In [ ]:
#####################################################
###Model 4

BACKBONE4 = 'seresnet34'
preprocess_input4 = sm.get_preprocessing(BACKBONE4)

# preprocess input
X_train4 = preprocess_input4(X_train)
X_test4 = preprocess_input4(X_test)


# define model
model4 = sm.Unet(BACKBONE4, encoder_weights='imagenet', classes=n_classes, activation=activation)

# compile keras model with defined optimozer, loss and metrics
model4.compile(optim, total_loss, metrics)



#print(model3.summary())

history4=model4.fit(X_train4, 
          y_train_cat,
          batch_size=1, 
          epochs=500,
          verbose=1,
          callbacks=[csv_logger4, reduce_lr, earlyStopping],
          validation_data=(X_test4, y_test_cat))


model4.save('/content/gdrive/MyDrive/Test_Output/Ensemble_Wanner/Seresnet-N.hdf5')


In [ ]:
#####################################################
###Model 5

BACKBONE5 = 'efficientnetb4'

preprocess_input5 = sm.get_preprocessing(BACKBONE5)

# preprocess input
X_train5 = preprocess_input5(X_train)
X_test5 = preprocess_input5(X_test)


# define model
model5 = sm.Unet(BACKBONE5, encoder_weights='imagenet', classes=n_classes, activation=activation)

# compile keras model with defined optimozer, loss and metrics
model5.compile(optim, total_loss, metrics)



#print(model3.summary())

history5=model5.fit(X_train5, 
          y_train_cat,
          batch_size=1, 
          epochs=500,
          verbose=1,
          callbacks=[csv_logger5, reduce_lr, earlyStopping],
          validation_data=(X_test5, y_test_cat))


model5.save('/content/gdrive/MyDrive/Test_Output/Ensemble_Wanner/EfficiennetB4-N.hdf5')


In [ ]:

#plot the training and validation accuracy and loss at each epoch
loss = history1.history['loss']
val_loss = history1.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history1.history['iou_score']
val_acc = history1.history['val_iou_score']

plt.plot(epochs, acc, 'y', label='Training IOU')
plt.plot(epochs, val_acc, 'r', label='Validation IOU')
plt.title('Training and validation IOU')
plt.xlabel('Epochs')
plt.ylabel('IOU')
plt.legend()
plt.show()

In [ ]:
#####Loading previously trained models ###################
####Can be expanded to load 5 models at the same time ################
##### load best three performing models##############

###Model 1
BACKBONE1 = 'resnet34'


preprocess_input1 = sm.get_preprocessing(BACKBONE1)

# preprocess input
X_train1 = preprocess_input1(X_train)
X_test1 = preprocess_input1(X_test)

############################################################
###Model 2

BACKBONE2 = 'inceptionv3'

preprocess_input2 = sm.get_preprocessing(BACKBONE2)

# preprocess input
X_train2 = preprocess_input2(X_train)
X_test2 = preprocess_input2(X_test)

#####################################################
###Model 3

BACKBONE3 = 'vgg19'

preprocess_input3 = sm.get_preprocessing(BACKBONE3)

# preprocess input
X_train3 = preprocess_input3(X_train)
X_test3 = preprocess_input3(X_test)

#####################################################




In [ ]:
#from keras.models import load_model


model1 = tf.keras.models.load_model('/content/gdrive/MyDrive/Test_Output/Ensemble_PrHu/resnet34-AUG.hdf5', compile=False)
model2 = tf.keras.models.load_model('/content/gdrive/MyDrive/Test_Output/Ensemble_PrHu/INception-AUG.hdf5', compile=False)
model3 = tf.keras.models.load_model('/content/gdrive/MyDrive/Test_Output/Ensemble_PrHu/VGG19-AUG.hdf5', compile=False)
#model3 = tf.keras.models.load_model('/content/gdrive/MyDrive/saved_models_20/EfficientnetB4_BONcell6_6k_Aug.hdf5', compile=False)

#Weighted average ensemble
models = [model1, model2, model3]
#preds = [model.predict(X_test) for model in models]

pred1 = model1.predict(X_test1)
pred2 = model2.predict(X_test2)
pred3 = model3.predict(X_test3)

preds=np.array([pred1, pred2, pred3])

#preds=np.array(preds)
weights = [0.4, 0.4, 0.2]
#weights = [0.15, 0.7, 0.15]

#Use tensordot to sum the products of all elements over specified axes.
weighted_preds = np.tensordot(preds, weights, axes=((0),(0)))
weighted_ensemble_prediction = np.argmax(weighted_preds, axis=3)

y_pred1_argmax=np.argmax(pred1, axis=3)
y_pred2_argmax=np.argmax(pred2, axis=3)
y_pred3_argmax=np.argmax(pred3, axis=3)


In [ ]:
# individual model prediction
n_classes = 3
IOU1 = tf.keras.metrics.MeanIoU(num_classes=n_classes)  
IOU2 = tf.keras.metrics.MeanIoU(num_classes=n_classes)  
IOU3 = tf.keras.metrics.MeanIoU(num_classes=n_classes) 
IOU_weighted = tf.keras.metrics.MeanIoU(num_classes=n_classes)  

IOU1.update_state(y_test[:,:,:,0], y_pred1_argmax)
IOU2.update_state(y_test[:,:,:,0], y_pred2_argmax)
IOU3.update_state(y_test[:,:,:,0], y_pred3_argmax)
IOU_weighted.update_state(y_test[:,:,:,0], weighted_ensemble_prediction)


print('IOU Score for model1 = ', IOU1.result().numpy())
print('IOU Score for model2 = ', IOU2.result().numpy())
print('IOU Score for model3 = ', IOU3.result().numpy())
print('IOU Score for weighted average ensemble = ', IOU_weighted.result().numpy())

In [ ]:
#Individual class prediction
values = np.array(IOU_weighted.get_weights()).reshape(n_classes, n_classes)
print(values)
class1_IoU = values[0,0]/(values[0,0] + values[0,1] + values[0,2] + values[1,0]+ values[2,0])
class2_IoU = values[1,1]/(values[1,1] + values[1,0] + values[1,2] + values[0,1]+ values[2,1])
class3_IoU = values[2,2]/(values[2,2] + values[2,0] + values[2,1] + values[0,2]+ values[1,2])


print("IoU for class1 is: ", class1_IoU)
print("IoU for class2 is: ", class2_IoU)
print("IoU for class3 is: ", class3_IoU)


plt.imshow(train_images[0, :,:,0], cmap='gray')
plt.imshow(train_masks[0], cmap='gray')

In [ ]:
###########################################
#Grid search for the best combination of w1, w2, w3 that gives maximum acuracy

import pandas as pd
df = pd.DataFrame([])

for w1 in range(0,3):
    for w2 in range(0,3):
        for w3 in range(0,3):
            wts = [w1/10.,w2/10.,w3/10]
            
            IOU_wted = tf.keras.metrics.MeanIoU(num_classes=n_classes) 
            wted_preds = np.tensordot(preds, wts, axes=((0),(0)))
            wted_ensemble_pred = np.argmax(wted_preds, axis=3)
            IOU_wted.update_state(y_test[:,:,:,0], wted_ensemble_pred)
            print("Now predciting for weights :", w1/10., w2/10., w3/10., " : IOU = ", IOU_wted.result().numpy())
            df = df.append(pd.DataFrame({'wt1':wts[0],'wt2':wts[1], 
                                         'wt3':wts[2], 'IOU': IOU_wted.result().numpy()}, index=[0]), ignore_index=True)
            
max_iou_row = df.iloc[df['IOU'].idxmax()]
print("Max IOU of ", max_iou_row[3], " obained with w1=", max_iou_row[0],
      " w2=", max_iou_row[1], " and w3=", max_iou_row[2])         


In [ ]:
#############################################################
opt_weights = [max_iou_row[0], max_iou_row[1], max_iou_row[2]]

#Use tensordot to sum the products of all elements over specified axes.
opt_weighted_preds = np.tensordot(preds, opt_weights, axes=((0),(0)))
opt_weighted_ensemble_prediction = np.argmax(opt_weighted_preds, axis=3)

In [ ]:
#######################################################
#Predict on a few images

#import random
#test_img_number = random.randint(0, len(X_test))
#test_img = X_test[test_img_number]
test_img_number = 10
test_img = X_test[test_img_number]
ground_truth=y_test[test_img_number]
test_img_norm=test_img[:,:,:]
test_img_input=np.expand_dims(test_img_norm, 0)

#Weighted average ensemble
models = [model1, model2, model3]

test_img_input1 = preprocess_input1(test_img_input)
test_img_input2 = preprocess_input2(test_img_input)
test_img_input3 = preprocess_input3(test_img_input)

test_pred1 = model1.predict(test_img_input1)
test_pred2 = model2.predict(test_img_input2)
test_pred3 = model3.predict(test_img_input3)

test_preds=np.array([test_pred1, test_pred2, test_pred3])

#Use tensordot to sum the products of all elements over specified axes.
weighted_test_preds = np.tensordot(test_preds, opt_weights, axes=((0),(0)))
weighted_ensemble_test_prediction = np.argmax(weighted_test_preds, axis=3)[0,:,:]


plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[:,:,0], cmap='gray')

plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth[:,:,0], cmap='jet')

plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(weighted_ensemble_test_prediction, cmap='jet')

plt.show()



#####################################################################
